In [ ]:
!pip install git+https://github.com/keras-team/keras-tuner.git
!pip install autokeras

In [6]:
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import autokeras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [4]:
def get_wav(path):
    '''
    Load wav file from disk and down-samples to RATE
    '''

    y, sr = librosa.load(directory1+'/'+path)
    return(librosa.core.resample(y=y,orig_sr=sr,target_sr=sr, scale=True),sr)

lis = []
directory1 = 'C:/Users/Ahmed/Desktop/dissertation/archive/recordings/recordings_wave'
for filename in os.listdir(directory1):
    if filename.endswith("mp3.wav"):
        step = get_wav(filename)[:40000]
        lis.append(step)
        
Audio_data = pd.Dataframe(lis)
Audio_data.to_csv("wave.csv", sep=',',index=False)

KeyboardInterrupt: 

In [ ]:
Audio = pd.read_csv('../input/wave-data/wave.csv')
labels_binary = pd.read_csv('../input/labels/labels.csv')
labels_multi = pd.read_csv('../input/labels/labels_5.csv')
                           
def to_mfcc(wav):

    return(librosa.feature.mfcc(y=wav, sr=22050))

mfccs = []
for i in range(len(Audio)):
    mfcc = to_mfcc(np.array(Audio.iloc[i]))  
    mfccs.append(mfcc)

In [7]:
## GPU needed for AutoKeras
device_name = tf.test.gpu_device_name()
assert device_name == '/device:GPU:0', 'GPU device not found'
tf.config.list_physical_devices('GPU')

AssertionError: GPU device not found

## BINARY MODELS USING AUTOKERAS

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_binary)
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols)
class_weights = {0: 0.38564555,
                1: 1.84665227}

input_node = .Input()
output_node = autokeras.DenseBlock()(input_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols )
y_test = y_test.astype(float)
predictions =  np.round(model.predict(np.array(X_test).astype(float)))
    
print('test accuracy = ',accuracy_score(y_test, np.asarray(predictions)))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))
cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['English ', 'None English ']); ax.yaxis.set_ticklabels(['English ', 'None English ']);

model.summary()

model.save('autokeras_binary_MLP.h5')

In [ ]:
model.save('autokeras_binary_MLP_FINAL.h5')

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_binary)
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols,1)
class_weights = {0: 0.38564555,
                1: 1.84665227}

input_node = .Input()
output_node = autokeras.ConvBlock()(input_node)
output_node = autokeras.DenseBlock()(output_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols )
y_test = y_test.astype(float)
predictions =  np.round(model.predict(np.array(X_test).astype(float)))
    
print('test accuracy = ',accuracy_score(y_test, np.asarray(predictions)))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))
cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['None English ', 'English ']); ax.yaxis.set_ticklabels(['None English', 'English ']);

model.summary()

model.save('autokeras_binary_CNN.h5')

In [ ]:

model.save('autokeras_binary_CNN_FINAL.h5')

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_binary)
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols)

class_weights = {0: 0.38564555,
                1: 1.84665227}

input_node = autokeras.Input()
output_node = autokeras.RNNBlock(bidirectional=False)(input_node)
output_node = autokeras.DenseBlock()(output_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols )
y_test = y_test.astype(float)
predictions =  np.round(model.predict(np.array(X_test).astype(float)))
    
print('test accuracy = ',accuracy_score(y_test, np.asarray(predictions)))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))
cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['None English ', 'English ']); ax.yaxis.set_ticklabels(['None English', 'English ']);

model.summary()

model.save('autokeras_binary_RNN.h5')

In [ ]:
model.save('autokeras_binary_RNN_FINAL.h5')

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_binary)
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols)
class_weights = {0: 0.38564555,
                1: 1.84665227}

input_node = autokeras.Input()
output_node = autokeras.ConvBlock()(input_node)
output_node = autokeras.RNNBlock()(output_node)
output_node = autokeras.DenseBlock()(output_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols )
y_test = y_test.astype(float)
predictions =  np.round(model.predict(np.array(X_test).astype(float)))
    
print('test accuracy = ',accuracy_score(y_test, np.asarray(predictions)))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))
cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['None English ', 'English ']); ax.yaxis.set_ticklabels(['None English', 'English ']);

model.summary()


model.save('autokeras_binary_Transformer.h5')

In [ ]:
model.save('autokeras_binary_Transformer_FINAL.h5')

 ## MULTI - CLASS MODELS USING AUTOKERAS

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_multi)


# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols)
# class_weights = {0: 0.30546624,
#                     1: 0.61555076,
#                     2: 2.19230769,
#                     3: 3.47560976,
#                     4: 5.48076923,
#                     5: 5.7 }

input_node = autokeras.Input()
output_node = autokeras.DenseBlock()(input_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val) )

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols, 1 )
y_test = y_test.astype(float)
predictions =  np.argmax(model.predict(np.array(X_test).astype(float)), axis=1)



print('test accuracy = ',accuracy_score(y_test, predictions))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))

cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Arabic', 'English','French', 'Mandarin','Other','Spanish']); ax.yaxis.set_ticklabels(['Arabic', 'English','french', 'mandarin','other','spanish'], rotation=45);


model.summary()

model.save('autokeras_multi_MLP.h5')

In [ ]:
model.save('autokeras_multi_MLP_FINAL.h5')
# other = 4
#arabic  = 0
# english =1
#spanish =5
# mandarin = 3
# french =2

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_multi)


# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols)
# class_weights = {0: 0.30546624,
#                     1: 0.61555076,
#                     2: 2.19230769,
#                     3: 3.47560976,
#                     4: 5.48076923,
#                     5: 5.7 }

input_node = autokeras.Input()
output_node = autokeras.ConvBlock()(input_node)
output_node = autokeras.DenseBlock()(output_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val) )

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols, 1 )
y_test = y_test.astype(float)
predictions =  np.argmax(model.predict(np.array(X_test).astype(float)), axis=1)



print('test accuracy = ',accuracy_score(y_test, predictions))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))

cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Arabic', 'English','French', 'Mandarin','Other','Spanish']); ax.yaxis.set_ticklabels(['Arabic', 'English','french', 'mandarin','other','spanish'], rotation=45);


model.summary()

model.save('autokeras_multi_CNN.h5')

In [ ]:
model.save('autokeras_multi_CNN_FINAL.h5')

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_multi)


# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols)
# class_weights = {0: 0.30546624,
#                     1: 0.61555076,
#                     2: 2.19230769,
#                     3: 3.47560976,
#                     4: 5.48076923,
#                     5: 5.7 }

input_node = autokeras.Input()
output_node = autokeras.RNNBlock()(input_node)
output_node = autokeras.DenseBlock()(output_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val) )

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols, 1 )
y_test = y_test.astype(float)
predictions =  np.argmax(model.predict(np.array(X_test).astype(float)), axis=1)



print('test accuracy = ',accuracy_score(y_test, predictions))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))

cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Arabic', 'English','French', 'Mandarin','Other','Spanish']); ax.yaxis.set_ticklabels(['Arabic', 'English','french', 'mandarin','other','spanish'], rotation=45);


model.summary()

model.save('autokeras_multi_RNN.h5')

In [ ]:
model.save('autokeras_multi_RNN_FINAL.h5')

In [ ]:

X = np.asarray(mfccs).astype('float32')
y = LabelEncoder().fit_transform(labels_multi)


# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle = True, stratify = y_train)

rows = np.array(X_train)[0].shape[0]
cols = np.array(X_train)[0].shape[1]

X_train = X_train.reshape(X_train.shape[0], rows,cols)
class_weights = {0: 0.30546624,
                    1: 0.61555076,
                    2: 2.19230769,
                    3: 3.47560976,
                    4: 5.48076923,
                    5: 5.7 }

input_node = .Input()
output_node = autokeras.ConvBlock()(input_node)
output_node = autokeras.RNNBlock()(output_node)
output_node = autokeras.DenseBlock()(output_node)
output_node = autokeras.ClassificationHead()(output_node)


clf = autokeras.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
history= clf.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val)  )

plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])
plt.title('English vs None-English accent accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


loss, acc = clf.evaluate(X_test, y_test, verbose=True)
print('Accuracy: %.3f' % acc)
model = clf.export_model()

rows = np.array(X_test[0]).shape[0]
cols = np.array(X_test[0]).shape[1]
X_test = np.asarray(X_test).reshape(np.asarray(X_test).shape[0], rows, cols, 1 )
y_test = y_test.astype(float)
predictions =  np.argmax(model.predict(np.array(X_test).astype(float)), axis=1)



print('test accuracy = ',accuracy_score(y_test, predictions))
print('test precision = ',precision_score(y_test, predictions, average='weighted'))
print('test recall = ',recall_score(y_test, predictions, average='weighted'))
print('test f1 = ',f1_score(y_test, predictions, average='macro'))
print('matthsew correlation = ',matthews_corrcoef(y_test, predictions))

cf = confusion_matrix(y_test, predictions)
ax= plt.subplot()
sns.heatmap(cf, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Arabic', 'English','French', 'Mandarin','Other','Spanish']); ax.yaxis.set_ticklabels(['Arabic', 'English','french', 'mandarin','other','spanish'], rotation=45);


model.summary()

model.save('autokeras_multi_EncoderDecoder2.h5')

In [ ]:
model.save('autokeras_multi_EncoderDecoder_FINAL.h5')